In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import discrete

import re
import pandas as pd
import math 
import csv
import time
import dateutil
from datetime import datetime
import seaborn as sns

C:\Users\nolefp\Anaconda\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
pd.options.display.float_format = '{:,.2f}'.format
sns.set_style("whitegrid")
sns.set_context("poster")

In [5]:
# Matplotlib Formatting
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import ticker
"""
mpl.rcParams["axes.grid"]=False
mpl.rcParams["xtick.major.size"] = 4
mpl.rcParams["xtick.minor.size"] = 2
mpl.rcParams["xtick.major.width"] = 0.5 
mpl.rcParams["xtick.minor.width"] = 0.5 

mpl.rcParams["ytick.major.size"] = 4
mpl.rcParams["ytick.minor.size"] = 2
mpl.rcParams["ytick.major.width"] = 0.5 
mpl.rcParams["ytick.minor.width"] = 0.5 
"""

millnames = ['',' Thousand',' Million',' Billion',' Trillion']
def millify(n, pos):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))
    thingtoreturn = n / 10**(3 * millidx)
    if thingtoreturn % 1 == 0:
        return '{:.0f}{}'.format(thingtoreturn, millnames[millidx])
    elif thingtoreturn % 0.1 == 0:
        return '{:.1f}{}'.format(thingtoreturn, millnames[millidx])
    else:
        return '{:.2f}{}'.format(thingtoreturn, millnames[millidx])

In [6]:
from IPython.core.display import HTML
HTML("<style>.container {width:50% !important; }</style>")

In [7]:
zipcodesdf = pd.read_excel("./General Info/zip_codes.xls", 0)

In [8]:
# change column names
zipcodesdf.columns = ["mun_id","mun","place","place_id","zip_name","zip_id"]

# drop unecessary columns
zipcodesdf.drop(["mun","place","zip_name"], axis=1, inplace=1)

# get ride of null zip code entries
zipcodesdf = zipcodesdf[zipcodesdf.zip_id.notnull()].copy()

# get rid of random zip doe without anything else
random = zipcodesdf[zipcodesdf.mun_id.isnull() & zipcodesdf.zip_id.notnull()].index
zipcodesdf.drop(random, inplace=1)

zipcodesdf = zipcodesdf.dropna()

In [9]:
for col in zipcodesdf:
    if re.search("id", col):
        zipcodesdf[col] = zipcodesdf[col].astype(int)
zipcodesdf.info()
zipcodesdf.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1137 entries, 2 to 6351
Data columns (total 3 columns):
mun_id      1137 non-null int32
place_id    1137 non-null int32
zip_id      1137 non-null int32
dtypes: int32(3)
memory usage: 22.2 KB


,mun_id,place_id,zip_id
2,70092,703494,11460
4,70092,703516,11461
5,70092,703524,11462
7,70092,703559,11427
11,70092,703591,11426


In [10]:
# save file in exports
zipcodesdf.to_excel("./exports/zip_codes.xlsx")

In [11]:
# get zip code monthly municipal panel
mpaneldf = pd.read_csv("./exports/mpaneldf.csv").drop("Unnamed: 0", axis=1)

In [15]:
# make kostolac align
zipcodesdf.loc[zipcodesdf[zipcodesdf["zip_id"] == 12208].index, 
               "mun_id"] = mpaneldf[mpaneldf.mun == "Kostolac"].mun_id.values[0]

In [16]:
# merge datasets
mpaneldf_zip = zipcodesdf.merge(mpaneldf, how="inner", on="mun_id")

In [14]:
mpaneldf_zip.to_csv("./exports/mpaneldf_zip.csv")